In [2]:
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

In [15]:
df = pd.read_csv('scraping_universe.csv')
df

,gvkey,splticrm,datadate,cik,fic,gsector,loc,tic
0,1004,BBB,01/31/2000,1750.0,USA,20.0,USA,AIR
1,1004,BBB,02/29/2000,1750.0,USA,20.0,USA,AIR
2,1004,BBB,03/31/2000,1750.0,USA,20.0,USA,AIR
3,1004,BBB,04/30/2000,1750.0,USA,20.0,USA,AIR
4,1004,BBB,05/31/2000,1750.0,USA,20.0,USA,AIR
...,...,...,...,...,...,...,...,...
329181,311524,B+,07/31/2014,1547063.0,USA,15.0,USA,TAM
329182,311524,B+,08/31/2014,1547063.0,USA,15.0,USA,TAM
329183,311524,B+,09/30/2014,1547063.0,USA,15.0,USA,TAM
329184,311524,B+,10/31/2014,1547063.0,USA,15.0,USA,TAM


In [17]:
list[df.gvkey.unique()]

list[array([  1004,   1010,   1034, ..., 266315, 285536, 311524], dtype=int64)]

In [8]:
data = pd.read_csv('All_Ratings.csv')
# print(len(data))
data = data[data['loc'] == 'USA']
# print(len(data))
data = data[data['fic'] == 'USA']
# print(len(data))
data = data.dropna(subset=['splticrm','cik','tic']) # Dropping all rows without ratings, ciks, or tickers
# print(len(data))
data

,gvkey,splticrm,datadate,cik,fic,gsector,loc,tic
102,1004,BBB,01/31/2000,1750.0,USA,20.0,USA,AIR
103,1004,BBB,02/29/2000,1750.0,USA,20.0,USA,AIR
104,1004,BBB,03/31/2000,1750.0,USA,20.0,USA,AIR
105,1004,BBB,04/30/2000,1750.0,USA,20.0,USA,AIR
106,1004,BBB,05/31/2000,1750.0,USA,20.0,USA,AIR
...,...,...,...,...,...,...,...,...
2064372,311524,B+,07/31/2014,1547063.0,USA,15.0,USA,TAM
2064373,311524,B+,08/31/2014,1547063.0,USA,15.0,USA,TAM
2064374,311524,B+,09/30/2014,1547063.0,USA,15.0,USA,TAM
2064375,311524,B+,10/31/2014,1547063.0,USA,15.0,USA,TAM


In [14]:
data.to_csv('scraping_universe.csv',index=False)

In [11]:
data.cik.nunique()

3420

In [9]:
data.isna().sum()

gvkey         0
splticrm      0
datadate      0
cik           0
fic           0
gsector     227
loc           0
tic           0
dtype: int64

In [8]:
#If you want to see what was filtered out before rating analysis
data = pd.read_csv('All_Ratings.csv')
#data = data.fillna(value = {"gsector": 'None', "splticrm": 'None'})

missing_rating = data[data['splticrm']=='None']
#missing_rating.to_csv('missing_rating.csv',index=False)
#missing_rating

missing_sector = data[data['gsector']=='None']
#missing_sector.to_csv('missing_sector.csv',index=False)
#missing_sector

non_usa = data[data['loc'] != 'USA']
non_usa = non_usa[non_usa['fic'] != 'USA']
#non_usa.to_csv('non_usa.csv',index=False)
#non_usa

In [9]:
#List of all ratings and fallen angel status for firms with more than 18 months of data ~ 6 filings
data = pd.read_csv('All_Ratings.csv')
data = data[data['loc'] == 'USA']
data = data[data['fic'] == 'USA']
data = data.dropna()

IG = ['AAA','AA+', 'AA', 'AA-','A+','A','A-','BBB+', 'BBB']
Sub = ['BBB-','BB+','BB','BB-','B+','B','B-','CCC+','CCC','CCC-','CC','D','SD']

IG_df = data[data['splticrm'].isin(IG)]
IG_df['Sub_IG'] = 0
Sub_df = data[data['splticrm'].isin(Sub)]
Sub_df['Sub_IG'] = 1

data = IG_df.append(Sub_df)
data['Sub_IG_p'] = data.groupby('gvkey')['Sub_IG'].shift(1)
data = data.dropna()

data['Fallen'] = data['Sub_IG'] - data['Sub_IG_p']
data = data.drop(columns=['Sub_IG','Sub_IG_p','fic', 'loc'])
data = data.groupby('gvkey').filter(lambda x : len(x)>18)
data

,gvkey,splticrm,datadate,cik,gsector,tic,Fallen
103,1004,BBB,02/29/2000,1750.0,20.0,AIR,0.0
104,1004,BBB,03/31/2000,1750.0,20.0,AIR,0.0
105,1004,BBB,04/30/2000,1750.0,20.0,AIR,0.0
106,1004,BBB,05/31/2000,1750.0,20.0,AIR,0.0
107,1004,BBB,06/30/2000,1750.0,20.0,AIR,0.0
...,...,...,...,...,...,...,...
2064372,311524,B+,07/31/2014,1547063.0,15.0,TAM,0.0
2064373,311524,B+,08/31/2014,1547063.0,15.0,TAM,0.0
2064374,311524,B+,09/30/2014,1547063.0,15.0,TAM,0.0
2064375,311524,B+,10/31/2014,1547063.0,15.0,TAM,0.0


In [10]:
#Unique companies that existed past 2000
df = data.groupby(['gvkey', 'cik','tic']).last() # Can change to first()
df = df.reset_index()
df['datadate'] = pd.to_datetime(df['datadate'])
df['Year'] = df['datadate'].dt.year
df = df[df['Year'] >= 2000]
df

,gvkey,cik,tic,splticrm,datadate,gsector,Fallen,Year
0,1004,1750.0,AIR,BB+,2017-02-28,20.0,0.0,2017
1,1010,910627.0,4165A,B+,2010-03-31,20.0,0.0,2010
2,1034,730469.0,ALO.2,B+,2009-01-31,35.0,0.0,2009
3,1045,6201.0,AAL,BB-,2017-02-28,20.0,0.0,2017
4,1048,65695.0,4267A,B+,2007-02-28,10.0,0.0,2007
...,...,...,...,...,...,...,...,...
2870,265188,1180638.0,WYNN1,BB-,2017-02-28,25.0,0.0,2017
2871,266163,1092839.0,DUNRQ,CC,2011-11-30,10.0,0.0,2011
2872,266257,1278752.0,AINV,BBB-,2017-02-28,40.0,0.0,2017
2873,266315,1286043.0,KRG,BBB-,2017-02-28,60.0,0.0,2017


In [11]:
# 10-Energy, 15-Materials, 20-Industrials, 25-Consumer Discretionary, 30-Consumer Staples, 35-Health Care, 40-Financials
    #45-IT, 50-Communication, 55-Utilities, 60-Real Estate
    
df['gsector'].value_counts()

25.0    522
40.0    391
20.0    371
10.0    299
55.0    258
45.0    221
15.0    208
35.0    207
50.0    153
30.0    142
60.0    103
Name: gsector, dtype: int64

In [12]:
#Get list of companies by sector for scraping
def get_companies(data, sector):
    data = data[data['gsector']==sector]
    companies = data['tic'].unique().tolist()
    return companies

In [13]:
test = get_companies(data, 10)
len(test)

299

In [14]:
#All instances of Fallen Angels between 2000-2017
fallen = data[data['Fallen'] == 1]
fallen = fallen.drop(columns=['Fallen'])
fallen['datadate'] = pd.to_datetime(fallen['datadate'])
fallen['Year'] = fallen['datadate'].dt.year
fallen

,gvkey,splticrm,datadate,cik,gsector,tic,Year
126,1004,BBB-,2002-01-31,1750.0,20.0,AIR,2002
1479,1048,BB,2002-12-31,65695.0,10.0,4267A,2002
2295,1075,BBB-,2005-12-31,764622.0,55.0,PNW,2005
5648,1164,BB,2002-05-31,723527.0,50.0,MCIP,2002
7348,1213,BBB-,2001-04-30,3000.0,20.0,ABF,2001
...,...,...,...,...,...,...,...
1979157,183257,BBB-,2009-07-31,1457543.0,35.0,CFN,2009
2014750,186278,BBB-,2013-08-31,1487712.0,20.0,AL,2013
2026331,187690,BBB-,2011-07-31,1360604.0,60.0,HTA,2011
2028163,189490,BBB-,2012-09-30,1524471.0,20.0,XLS,2012


In [15]:
fallen['gsector'].value_counts()

40.0    103
55.0     89
25.0     74
20.0     71
10.0     52
15.0     46
45.0     43
35.0     37
60.0     36
50.0     23
30.0     20
Name: gsector, dtype: int64

In [16]:
fallen['Year'].value_counts()

2000    178
2002     58
2001     55
2003     45
2005     36
2009     35
2007     32
2008     30
2006     24
2004     22
2010     18
2013     17
2011     11
2014      9
2016      9
2012      8
2015      6
2017      1
Name: Year, dtype: int64